In [ ]:
# 라이브러리 불러오기 및 함수화
import re
import pandas as pd

from math import log  # IDF 계산을 위해 필요
from konlpy.tag import Komoran
from pykospacing import Spacing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

komoran = Komoran()
spacing = Spacing()

In [ ]:
df = pd.read_excel('../../../../../data/filtered_news.xlsx')
# df = df[:][0:100]

# 기본 불용어 불러오기
korean_stopwords_path = '../../../../../data/stopwords-ko.txt'
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
# 텍스트 전처리 함수
def preprocessing(text):
    text = spacing(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# komoran토큰화 &불용어 처리 함수
def remove_stopwords(text, stopwords):
    tokens = []
    morphs = komoran.morphs(text)
    for token in morphs:
        if token not in stopwords:
            tokens.append(token)
    return tokens

# 텍스트 전처리 및 토큰화, 불용어 처리
cleaned_data = []
for i in range(len(df)):
    feature_text = df.loc[i, 'feature']
    processed_text = preprocessing(feature_text)
    cleaned_text = remove_stopwords(processed_text, stopwords)
    cleaned_text = ' '.join(cleaned_text)
    cleaned_data.append(cleaned_text)
df['cleaned'] = cleaned_data

In [ ]:
# 사이킷런을 이용한 TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['cleaned'])
vector_text = tfidf_matrix.toarray().tolist()
df['vectored'] = vector_text